In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [1]:
!pip install transformers transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [2]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate


Found existing installation: transformers 4.46.3
Uninstalling transformers-4.46.3:
  Successfully uninstalled transformers-4.46.3
Found existing installation: accelerate 1.1.1
Uninstalling accelerate-1.1.1:
  Successfully uninstalled accelerate-1.1.1
  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
  Using cached accelerate-1.1.1-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.46.3-py3-none-any.whl (10.0 MB)
Using cached accelerate-1.1.1-py3-none-any.whl (333 kB)


In [3]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
import pandas as pd


from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
!pip install evaluate
import evaluate
metric = evaluate.load("accuracy")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [6]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip summarizer-data.zip

--2024-11-28 10:44:06--  https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip [following]
--2024-11-28 10:44:06--  https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip.3’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.1s    

2024-11-28 10:44:06 (59.1 MB/s) - ‘summarizer-data.zip.3’ saved [7903594

In [ ]:
# from datasets import load_dataset

# dataset_samsum = load_dataset("knkarthick/samsum")

dataset_samsum = load_from_disk('samsum_dataset')
dataset_samsum

In [ ]:
dataset_samsum

In [ ]:
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]
print(f'Split lengthsL {split_lengths}')
print(f"Features:'{dataset_samsum['train'].column_names}")
print("\nDialogue:")
print(dataset_samsum['test'][1]["dialogue"])
print("\nSummary:")
print(dataset_samsum['test'][1]["summary"])

In [ ]:
def convert_examples_to_features(example_batch, tokenizer=tokenizer):
    # Convert list of dialogues/summaries to lists of encoded inputs
    input_encodings = tokenizer(
        example_batch['dialogue'],  # Pass the list of dialogues
        max_length=1024,
        truncation=True,
        padding="max_length",  # Add padding to handle varying lengths in a batch
    )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(
            example_batch['summary'],  # Pass the list of summaries
            max_length=128,
            truncation=True,
            padding="max_length",  # Add padding to handle varying lengths in a batch
        )

    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [ ]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True, fn_kwargs={'tokenizer': tokenizer})

In [ ]:

dataset_samsum_pt["train"]

In [ ]:
from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)


In [ ]:
from transformers import TrainingArguments, Trainer
trainer_args = TrainingArguments(
    output_dir = "pegasus-samsum",
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy = 'steps',
    eval_steps =500,
    save_steps=1e6,
    gradient_accumulation_steps=16)
#responsible for taking input and creating the batches of it.


In [ ]:
trainer = Trainer(model=model_pegasus,
                  args= trainer_args,
                  tokenizer=tokenizer,
                  data_collator= seq2seq_data_collator,
                  train_dataset= dataset_samsum_pt['test'], # training data too big to train , will take huge amount of time , using test data to train
                  eval_dataset= dataset_samsum_pt['validation'])

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mann-agrawal17 (ishuman). Use `wandb login --relogin` to force relogin


In [ ]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list-of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i:i+batch_size]


def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                                batch_size=16,

                                device = device, column_text= "article",
                                column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total= len(article_batches)):
        inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding="max_length", return_tensors= 'pt')

        summaries = model.generate(input_ids=inputs['input-ids'].to(device),
                                   attention_mask=inputs['attention_mask'].to(device),
                                   length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that th model does not genrate sequences that are too long'''

        # Finally, we decode the generated texts,
        # replace the token, and add the decoded texts with the references to the metric.

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                              clean_up_tokenization_spaces=True) for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

        metric.add_batch(predictions= decoded_summaries, references= target_batch)

        # Finally compute and return the ROUGE scores.

    score = metric.compute()
    return score







In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
# rouge_metric = load_metric('rouge')
import evaluate
metric = evaluate.load("rouge")

In [ ]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size =2, column_text = "dialogue", column_summary = "summary")
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index = [f'pegasus'])



In [ ]:
## save model
model_pagasus.save_pretrained("pegasus-samsum-model"
)


In [ ]:
## Save tokenizer
tokenizer.save_pretrained('tokenizer')


In [ ]:
#load
tokenizer = AutoTokenizer.from_pretrained('/content/tokenizer')


In [ ]:
#Prediction
gen_kwargs = {"length_penalty": 0.8, 'num_beams':8 , 'max_length': 128}

sample_text = dataset_samsum['test'][0]['dialogue']
reference = dataset_samsum['test'][0]['summary']
pipe = pipeline('summarization', model= 'pegasus-samsum-model', tokenizer= tokenizer)
##
print("Dialogue:")
print(sample_text)
print("\nReference Summary:")
print(reference)

print("\n Model Summary:")

print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

